# Labelme关键点检测标注转COCO格式

同济子豪兄 呕心沥血写成

2023-3-9

2023-3-10

## 导入工具包

In [1]:
import os
import json

import numpy as np

## 删除系统自动生成的多余文件

建议在 Linux 系统中运行爬虫、划分训练集测试集代码

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

./__MACOSX


In [3]:
!find . -iname '.DS_Store'

./triangle_test/.DS_Store


In [4]:
!find . -iname '.ipynb_checkpoints'

./.ipynb_checkpoints
./crab-new/.ipynb_checkpoints


### 删除多余文件

In [6]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [8]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [9]:
!find . -iname '__MACOSX'

In [10]:
!find . -iname '.DS_Store'

In [11]:
!find . -iname '.ipynb_checkpoints'

## 进入labelme标注文件目录

In [12]:
labelme_folder = 'triangle_test'

In [13]:
os.chdir(labelme_folder)

In [14]:
os.listdir()

['MVIMG_20230331_082339.json',
 'MVIMG_20230331_082339.jpg',
 'MVIMG_20230331_082341.jpg',
 'MVIMG_20230331_082341.json']

## 示例-导入一个labelme格式的json标注文件

In [15]:
# with open('1.json', 'r', encoding='utf-8') as f:
#     labelme = json.load(f)

In [16]:
# labelme['shapes']

In [17]:
# labelme.keys()

In [18]:
# labelme['version']

In [19]:
# labelme['flags']

In [20]:
# 图像文件名
# labelme['imagePath']

In [21]:
# labelme['imageData']

In [22]:
# 图像高度
# labelme['imageHeight']

In [23]:
# 图像宽度
# labelme['imageWidth']

## 创建coco格式的字典

In [24]:
coco = {}

## info

In [25]:
# coco['info'] = {}
# coco['info']['description'] = 'Labelme2coco keypoint format script from Zihao'
# coco['info']['year'] = 2023
# coco['info']['date_created'] = '2023/03/09'

## categories

supercategory：框的类别（一般只有一个类别）

keypoints：节点类别排序，后续的关键点坐标展示顺序

skeleton：可有可无，可视化时需要

In [26]:
class_list = {'supercategory': 'sjb_rect',
              'id': 1,
              'name': 'sjb_rect',
              'keypoints': ['angle_30', 'angle_60', 'angle_90'], # 大小写敏感
              'skeleton':[[0,1],[0,2], [1,2]]
             }

In [27]:
coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

## 函数-处理单个labelme标注json文件

In [28]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出个体框

            # 个体框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取个体框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 测试一下函数的效果，处理单个labelme格式的json标注文件

In [29]:
with open('MVIMG_20230331_082339.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [30]:
process_single_json(labelme)

[{'category_id': 1,
  'segmentation': [[2184.78, 3513.04], [306.52, 1804.35], [1336.96, 1273.91]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [215, 1208, 2048, 2370],
  'area': 4853760,
  'num_keypoints': 3,
  'keypoints': [2202, 3526, 2, 315, 1791, 2, 1336, 1273, 2]}]

## images和annotations

In [31]:
IMG_ID = 0
ANN_ID = 0

In [32]:
# 遍历所有 labelme 格式的 json 文件
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            ## 提取图像元数据
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
            ## 提取框和关键点信息
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass
    
with open('../labelme2coco.json', 'w') as f:
    json.dump(coco, f, indent=2)

MVIMG_20230331_082339.json 已处理完毕
MVIMG_20230331_082341.json 已处理完毕


In [33]:
coco

{'categories': [{'supercategory': 'sjb_rect',
   'id': 1,
   'name': 'sjb_rect',
   'keypoints': ['angle_30', 'angle_60', 'angle_90'],
   'skeleton': [[0, 1], [0, 2], [1, 2]]}],
 'images': [{'file_name': 'MVIMG_20230331_082339.jpg',
   'height': 4000,
   'width': 3000,
   'id': 0},
  {'file_name': 'MVIMG_20230331_082341.jpg',
   'height': 4000,
   'width': 3000,
   'id': 1}],
 'annotations': [{'category_id': 1,
   'segmentation': [[2184.78, 3513.04], [306.52, 1804.35], [1336.96, 1273.91]],
   'iscrowd': 0,
   'image_id': 0,
   'id': 0,
   'bbox': [215, 1208, 2048, 2370],
   'area': 4853760,
   'num_keypoints': 3,
   'keypoints': [2202, 3526, 2, 315, 1791, 2, 1336, 1273, 2]},
  {'category_id': 1,
   'segmentation': [[384.78, 1326.09], [1506.52, 921.74], [2223.91, 2995.65]],
   'iscrowd': 0,
   'image_id': 1,
   'id': 1,
   'bbox': [341, 873, 1922, 2144],
   'area': 4120768,
   'num_keypoints': 3,
   'keypoints': [2202, 2952, 2, 402, 1326, 2, 1493, 913, 2]}]}

## 验证MS COCO格式的标注

In [34]:
!pip install pycocotools

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [35]:
from pycocotools.coco import COCO

my_coco = COCO('../labelme2coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
